[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/yuyan124/sd_tools/blob/main/kohya_dreambooth.ipynb)

In [ ]:
# GPU INFO
!nvidia-smi

In [ ]:
# 安装python，可选择自己需要的版本
!apt-get install python3.10
!python -m pip uninstall pips
!update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.10 2
!sudo apt-get install python3.10-distutils
!wget https://bootstrap.pypa.io/pip/get-pip.py
!python get-pip.py
!python --version

In [ ]:
!git clone https://github.com/kohya-ss/sd-scripts.git
!cd sd-scripts 
!pip install --upgrade -r requirements.txt 

!apt-get -y install -qq aria2
!pip install --pre -U xformers
!pip install -q --pre triton
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/11745 -d /content/models -o model.safetensors

In [ ]:
# 挂载 Google Drive
from google.colab import drive
drive.mount('/content/drive')
!mkdir /content/train/name
!cp -r /content/drive/MyDrive/ai/6_name /content/train/name/
!ls /content/train/name/6_name

In [ ]:
def run_script(s):
    ! {s}

def make_args(d):
    arguments = ""
    for k, v in d.items():
        if type(v) == bool:
            arguments += f"--{k} " if v else ""
        elif type(v) == str and v:
            arguments += f"--{k} \"{v}\" "
        elif v:
            arguments += f"--{k}={v} "
    return arguments

name = ""

arguments = {
    "enable_bucket": True,
    "pretrained_model" : "./sd-models/model.safetensors", # base model path | 底模路径
    "train_data_dir" : f"./train/{name}", # train dataset path | 训练数据集路径
    "output_dir" : "/content/drive/MyDrive/ai/outputs",
    "logging_dir" : "./logs",

    # Train related params | 训练相关参数
    "resolution" : "512,512", # image resolution w,h. 图片分辨率，宽,高。支持非正方形，但必须是 64 倍数。
    "newwork_module": "networks.lora",
    "max_train_epochs" : 20, # max train epoches | 最大训练 epoch
    "train_batch_size" : 1, # batch size
    "save_every_n_epochs" : 2, # save every n epochs | 每 N 个 epoch 保存一次
    "network_dim" : 32, # network dim | 常用 4~128，不是越大越好
    "network_alpha": 32, # network alpha | 常用与 network_dim 相同的值或者采用较小的值，如 network_dim的一半 防止下溢。默认值为 1，使用较小的 alpha 需要提升学习率。
    "clip_skip" : 2, # clip skip | 玄学 一般用 2
    "train_unet_only" : 0, # train U-Net only | 仅训练 U-Net，开启这个会牺牲效果大幅减少显存使用。6G显存可以开启
    "train_text_encoder_only" : 0, # train Text Encoder only | 仅训练 文本编码器
    "mixed_precision": "fp16",
    "save_precision": "fp16",
    "seed": "-1",
    "cache_latents": True,
    "prior_loss_weight": 1,
    "max_token_length": 255,
    "caption_extension": ".txt",
    "min_bucket_reso": 256,
    "max_bucket_reso": 1024,

    # Learning rate | 学习率
    "learning_rate" : "1e-4",
    "unet_lr" : "1e-4",
    "text_encoder_lr" : "1e-5",
    "lr_scheduler" : "cosine_with_restarts", # "linear", "cosine", "cosine_with_restarts", "polynomial", "constant", "constant_with_warmup"
    "lr_warmup_steps": 0,

    "xformers": True,
    "shuffile_caption": True,
    "use_8bit_adam": True,

    # Output settings | 输出设置
    "output_name" : name, # output model name | 模型保存名称
    "save_model_as" : "safetensors" # model save ext | 模型保存格式 ckpt, pt, safetensors
  
}

run_script(f"""
accelerate launch --num_cpu_threads_per_process=8 "train_network.py" {make_args(arguments)} --network_module=networks.lora 
""")